In [2]:


# Importing libraries 
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import PIL 
import tensorflow as tf 
import glob
from pathlib import Path
import keras_tuner as kt
  
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential 


In [3]:
data_dir = "/media/a/EXT4/Downlaods/archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = Path(os.path.join(data_dir, "train")  )
valid_dir = Path(os.path.join(data_dir, "valid"))
diseases = os.listdir(train_dir)
jpg_train_files = list(Path(train_dir).rglob('*/*.jpg'))
len(jpg_train_files) #70295

70295

In [4]:
## Modify as needed # This is used to batch rename '.JPG' to '.jpg' or vice versa if needed
# # Get a list of all jpg files in train_dir and its subdirectories
# jpg_train_files = list(train_dir.rglob('*.JPG'))

# # Rename each jpg file to JPG
# for jpg_file in jpg_train_files:
#     new_name = jpg_file.with_suffix('.jpg')  # Change the extension to lowercase
#     jpg_file.rename(new_name)

In [5]:
diseases

['Tomato___Spider_mites Two-spotted_spider_mite',
 'Apple___Cedar_apple_rust',
 'Soybean___healthy',
 'Pepper,_bell___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Grape___Esca_(Black_Measles)',
 'Blueberry___healthy',
 'Apple___healthy',
 'Tomato___Bacterial_spot',
 'Grape___Black_rot',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___Early_blight',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Tomato___Target_Spot',
 'Corn_(maize)___Common_rust_',
 'Strawberry___healthy',
 'Tomato___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Cherry_(including_sour)___healthy',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Apple___Black_rot',
 'Apple___Apple_scab',
 'Potato___healthy',
 'Tomato___Late_blight',
 'Squash___Powdery_mildew',
 'Cherry_(including_sour)___Powdery_mildew',
 'Corn_(maize)___healthy',
 'Potato___Early_blight',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Grape___healthy',
 'Tomato___Leaf_Mold',
 'Raspberry___healthy'

In [6]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=2))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=["accuracy"]) # loss="binary_crossentropy"
    
    return nn_model

In [7]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

2024-05-14 20:47:22.663311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
batch_size = 32
img_height = 256
img_width = 256

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 70295 files belonging to 38 classes.


In [10]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  valid_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 17572 files belonging to 38 classes.


In [11]:
class_names = train_ds.class_names

In [12]:
num_classes = 38

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])


In [13]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [14]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=2
)


Epoch 1/2
2197/2197 [==============================] - 869s 394ms/step - loss: 0.9084 - accuracy: 0.7297 - val_loss: 0.4392 - val_accuracy: 0.8590
Epoch 2/2
 634/2197 [=======>......................] - ETA: 7:29 - loss: 0.3508 - accuracy: 0.8907

In [ ]:
# Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

NameError: name 'X_train_scaled' is not defined